# Cuaderno 9: Interfaz Python de Gurobi - Introducción

Describiremos en este cuaderno el uso básico de la interfaz Python de Gurobi para crear y resolver un modelo de de programación lineal entera.

Implementaremos el siguiente modelo, tomado del *manual de Gurobi Optimization (2020)*
\begin{align*}
&\max x + y + 2z\\
&\mbox{s.r.}\\
& x + 2y + 3z \leq 4\\
& x + y \geq 1\\
& x,y,z \in \{0, 1\}
\end{align*}

Lo primero que debemos hacer es importar el módulo `gurobipy` con las clases y funciones de la interfaz de Gurobi. Usamos la forma `from <modulo> import *` para evitar tener que repetir el nombre del módulo en el resto del programa:

In [13]:
from gurobipy import *

Luego llamamos al constructor `Model(<nombre_del_modelo>)` para crear un nuevo objeto que representará al modelo de programación lineal entera.

In [14]:
m = Model("mip1")

La construcción, modificación y solución de este modelo se realiza a través de métodos de la clase Model.

Lo primero que debemos hacer es agregar variables al modelo. Para ello se puede emplear el método `addVar()`. Dos argumentos importantes que se pueden pasar a este método son `vtype`, que indica el tipo de la variable; y `name` que almacena su nombre como una cadena de caracteres.

Los valores posibles para `vtype` son `GRB.BINARY` (variables binarias), `GRB.INTEGER` (variables enteras no negativas) y `GRB.CONTINUOUS` (variables reales no negativas). 

In [15]:
# Definir a x,y,z, como variables binarias
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

El siguiente paso es construir la función objetivo del modelo llamando al método `setObjective`. Este método recibe dos parámetros: una *expresión lineal* con la función objetivo en sí, y una constante que indica el sentido de la optimización. Para el segundo parámetro debe pasarse uno de los dos valores: `GRB.MAXIMIZE` o `GRB.MINIMIZE`.

In [16]:
# Definir la función objetivo e indicar que se trata de maximizacion
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

Observar más de cerca el primer parámetro: la interfaz Python de Gurobi define un nuevo tipo de datos específico para construir **expresiones lineales**. Una expresión lineal se compone de una suma o resta de *términos lineales*. Cada término lineal puede ser una variable (ej: `x`), un número, o el producto de un número con una variable (ej: `2 * z`).

Una vez definida la función objetivo del modelo, el paso siguiente es agregar las restricciones llamando al método `addConstr()`:

In [17]:
# Agregar dos restricciones al modelo
m.addConstr(x + 2 * y + 3 * z <= 4, "c0")
m.addConstr(x + y >= 1, "c1")

<gurobi.Constr *Awaiting Model Update*>

El primer parámetro de la función `addConstr()` es una restricción lineal, el segundo parámetro es una cadena de caracteres que almacena el nombre de la restricción.

La interfaz Python de Gurobi define un nuevo tipo de datos específico para representar **restricciones lineales**. Una expresión de este tipo se construye a partir de una expresión lineal; seguida por uno de los tres operadores de comparación `<=`, `==`, o `>=`; seguida por un número o por una segunda expresión lineal.

Para resolver el modelo, se llama al método `optimize()`:

In [18]:
# Optimizar el modelo
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xf43f5bdf
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.10 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


Al llamar a `optimize()`, se invocan automáticamente las funciones de la biblioteca de Gurobi necesarias para resolver el modelo. Se muestra la salida de las mismas.

Una vez terminada la solución del modelo, puede consultarse el valor óptimo de la función objetivo (en caso de que el modelo haya sido resuelto hasta la optimalidad) consultando el valor de `objVal`. Por otra parte, el método `m.getVars()` retorna una lista con las variables del modelo, la misma que puede iterarse para consultar información como el valor de cada variable:

In [22]:
# Valor de la funcion objetivo
print('Valor optimo: {}'.format(m.objVal))
# Datos de las variables
print('Datos de las variables')
for v in m.getVars():
    print('Nombre: {}\t\tValor: {}'.format(v.varName, v.x))

Valor optimo: 3.0
Datos de las variables
Nombre: x		Valor: 1.0
Nombre: y		Valor: 0.0
Nombre: z		Valor: 1.0


## Código completo

Se reproduce a continuación el código completo del modelo anterior, tal como se encuentra documentado en el manual de Gurobi.

Observar que las instrucciones anteriores están contenidas dentro de un bloque `try` ... `except`. Esta es una práctica recomendada para el correcto manejo de las excepciones que pudieran producirse durante la construcción y solución del modelo.

In [31]:
#!/usr/bin/env python3.7

# Copyright 2020, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary

import gurobipy as gp
from gurobipy import GRB

try:

    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")
    # m.addConstr(x + 2 * y + 3 * z >= 5, "c2")

    # Add constraint: x + y >= 1
    m.addConstr(x + y  >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xf43f5bdf
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.09 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1
y 0
z 1
Obj: 3
